# **SAMPLING PROCEDURE**
**License : CC BY-NC-SA 4.0**

In this notebook, we demonstrate how to sample from our trained ddpm model.  
Source :  
- https://huggingface.co/blog/annotated-diffusion  
- https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/annotated_diffusion.ipynb#scrollTo=5153024b 

## **Jean Zay Environment**
We should use the `pytorch-gpu-2.1.1_py3.11.5` jupyter kernel.

In [ ]:
!module list

## **Import libraries**

In [ ]:
import torch

from code.model import Unet
from code.fashion_mnist_dataset import get_dataloader
from code.forward_diffusion import linear_beta_schedule, cosine_beta_schedule, get_alph_bet
from code.reverse_diffusion import train
from code.utils import show_images

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## **TRAINING MODEL**

In [ ]:
# ARGUMENTS
epochs = 3
batch_size = 64
T = 1000

# GET MODEL
model = Unet(   
    dim=28,
    init_dim=None,
    out_dim=None,
    dim_mults=(1, 2, 4),
    channels=1,
    with_time_emb=True,
    convnext_mult=2,
).to(DEVICE)

# GET DATALOADER
train_dataloader, channels, image_size, n_batch = get_dataloader(batch_size=batch_size,
                                                                 test=False)

# GET BETA SCHEDULE
constants_dict = get_alph_bet(T, schedule=cosine_beta_schedule)

# TRAIN MODEL
model = train(epochs, train_dataloader, model, T, constants_dict)

## **SAMPLING**

### **Definition of $ p_{\theta}(x_{t-1}|x_t) $**
Now we need a function to retrieve $x_{t-1}$ from $x_t$ and the predicted $z_t$. It corresponds to the reverse diffusion kernel:
![p_sample](img/reverse_diffusion_process.png)

#### **TODO**
**Complete p_sample which compute $x_{t-1}$ from $x_t$ and $z_t$.**

In [ ]:
@torch.no_grad()
def p_sample(constants_dict, batch_xt, predicted_noise, batch_t):
    """
    Solution in `code.reverse_diffusion.py`
    """
    # We first get every constants needed
    betas_t = None
    sqrt_one_minus_alphas_cumprod_t = None
    sqrt_recip_alphas_t = None
    
    # Equation 11 in the ddpm paper
    # Use predicted noise to predict the mean (mu theta)
    model_mean = None
    
    # We have to be careful to not add noise if we want to predict the final image
    predicted_image = torch.zeros(batch_xt.shape).to(batch_xt.device)
    t_zero_index = (batch_t == torch.zeros(batch_t.shape).to(batch_xt.device))
    
    # Algorithm 2 line 4, we add noise when timestep is not 0:
    posterior_variance_t = None
    noise = None
    
    # If t>0, we add noise to mu 
    predicted_image[~t_zero_index] = model_mean[~t_zero_index] + (
        torch.sqrt(posterior_variance_t[~t_zero_index]) * noise[~t_zero_index]
    ) 
    # If t=0, we don't add noise to mu
    predicted_image[t_zero_index] = model_mean[t_zero_index]
    
    return predicted_image

## **Define sampler with p_sample**
With the reverse diffusion process and a trained model, we can now make the sampling function corresponding to this algorithm:
![sampling](img/sampling_algorithm.png)

In [ ]:
# Algorithm 2 (including returning all images)
@torch.no_grad()
def sampling(model, shape, T, constants_dict):
    b = shape[0]
    # start from pure noise (for each example in the batch)
    batch_xt = None
    
    # create a vector with batch-size time the timestep
    batch_t = None
    batch_t = batch_t.type(torch.int64).to(DEVICE)
    
    imgs = []

    for t in tqdm(reversed(range(0, T)), desc='sampling loop time step', total=T):
        batch_t -= 1
        predicted_noise = None
        batch_xt = None
        
        imgs.append(batch_xt.cpu())
        
    return imgs

## **Check the sampling !**

In [ ]:
print("Check generation:")  
list_gen_imgs = sampling(model, (batch_size, channels, image_size, image_size), T, constants_dict)
show_images(list_gen_imgs[-1])